#  Lab Assignment Seven: Sequential Network Architectures

Name 1: Jadon Swearingen (5000)
Name 2: Ephraim Sun (7000)
Name 3: Adeeb Abdul Taher (7000)


In this lab, you will select a prediction task to perform on your dataset, evaluate a sequential architecture and tune hyper-parameters. If any part of the assignment is not clear, ask the instructor to clarify. 

### Dataset Selection

Dataset: https://www.kaggle.com/datasets/rajatkumar30/fake-news

We have chosen a fake news dataset where given the title and text of the article, we should predict if the news is fake or real. 

### Preparation

[1 points] Define and prepare your class variables. Use proper variable representations (int, float, one-hot, etc.). Use pre-processing methods (as needed). Describe the final dataset that is used for classification/regression (include a description of any newly formed variables you created). Discuss methods of tokenization in your dataset as well as any decisions to force a specific length of sequence.  


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import seaborn as sns
import nltk, re, string
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB, MultinomialNB, BernoulliNB
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score
from sklearn.model_selection import cross_val_score
from sklearn.datasets import make_classification
from sklearn.metrics import make_scorer, f1_score
from sklearn.metrics import accuracy_score, precision_score, recall_score

import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Input, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, SimpleRNN, LSTM, GRU, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import KFold

mpl.style.use("seaborn-v0_8-deep")
mpl.rcParams["figure.figsize"] = (20, 5)
mpl.rcParams["figure.dpi"] = 100
plt.rcParams["figure.dpi"] = 100
plt.rcParams["lines.linewidth"] = 2

In [ ]:
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.sentiment import SentimentIntensityAnalyzer

In [ ]:
nltk.download("vader_lexicon")
nltk.download("punkt")
nltk.download("stopwords")


In [ ]:
# Load dataset

df = pd.read_csv("./dataset/news.csv")

# Load less data
df =df.head(1000)


df

#### Preprocess Data

This function performs several preprocessing steps on the input DataFrame:
1. Converts the 'title' and 'text' columns to string type to ensure consistency.
2. Encodes the 'label' column, mapping 'FAKE' to 1 and 'REAL' to 0, to prepare for machine learning algorithms.
3. Drops the 'Unnamed: 0' column which is often an artifact from reading files with an index.
4. Combine the article text + article together
5. Clean the text by lowercasing everything, removing stopwords, and removing non-alphabetic characters.

In [ ]:
def cleanText(txt):
      
    #Creates list of possible stopwords from nltk library
    stop = stopwords.words('english')
    
    # Lowercase
    txt = txt.lower()
    
    # Remove stopwords
    txt = ' '.join([word for word in txt.split() if word not in (stop)])
    
    # Remove non-alphabetic characters
    txt = re.sub('[^a-z]',' ',txt)
    return txt  


def pre_process(df):
    """
    Parameters:
    - df (pd.Dataframe): The DataFrame to be preprocessed

    Returns: 
    - df (pd.DataFrame): The DataFrame after preprocessed

    """

    columns = ["title", "text"]
    for col in columns:
        df[col] = df[col].astype(str)  # Convert each column to string

    # Map 'label' values from 'FAKE'/'REAL' to 1/0
    df["label"] = df["label"].map({"FAKE": 1, "REAL": 0})

    # Remove the 'Unnamed: 0' column, if it exists, as it's usually an artifact
    df = df.drop(["Unnamed: 0"], axis=1)


    # Combining text and title for creating full news article with headline
    df['final_news'] = df['title'] + " " + df['text']

    # Cleaning the text
    df['final_news'] = df['final_news'].apply(cleanText)

    return df

In [ ]:
def num_characters(df, col_name):
    """
    Adds a new column to the DataFrame containing the number of characters in each message.

    Parameters:
    - df: DataFrame. The DataFrame to which the new column will be added.
    - col_name: str. The name of the column containing the messages.

    Returns:
    - DataFrame. The original DataFrame with a new column added.
    """

    df["Characters"] = df[col_name].apply(len)
    return df


def num_sentences(df, col_name):
    """
    Adds a new column to the DataFrame containing the number of sentences in each message.

    Parameters:
    - df: DataFrame. The DataFrame to which the new column will be added.
    - col_name: str. The name of the column containing the messages.

    Returns:
    - DataFrame. The original DataFrame with a new column added.
    """
    df["Sentences"] = df[col_name].apply(lambda x: len(nltk.sent_tokenize(x)))
    return df


def num_words(df, col_name):
    """
    Adds a new column to the DataFrame containing the number of words in each message.

    Parameters:
    - df: DataFrame. The DataFrame to which the new column will be added.
    - col_name: str. The name of the column containing the messages.

    Returns:
    - DataFrame. The original DataFrame with a new column added.
    """
    df["Words"] = df[col_name].apply(lambda x: len(nltk.word_tokenize(x)))
    return df


In [ ]:
def pie_plot(df, col_name):
    """
    Creates a pie plot to show the distribution of real and fake news in the DataFrame.

    Parameters:
    - df: DataFrame. The DataFrame containing the data.
    - col_name: str. The name of the column containing the labels.

    Returns:
    - None. Displays a pie plot.
    """
    sizes = df["label"].value_counts()
    labels = ["Real", "Fake"]
    colors = ["#ff9999", "#66b3ff"]

    plt.pie(
        sizes,
        autopct="%1.1f%%",
        colors=colors,
        startangle=90,
        explode=(0.1, 0),
        shadow=True,
    )

    plt.title("Distribution of Real and Fake News in " + col_name)
    plt.legend(labels, title="News Type", loc="center left", bbox_to_anchor=(1, 0.5))
    plt.show()

In [ ]:
def plot_distribution_features(df, features, col_name):
    """
    Plot distributions of specified features in a DataFrame, comparing 'Real' and 'Fake' news.

    Parameters:
    - df: DataFrame containing the data.
    - features: List of strings, names of the columns for which to plot the distributions.

    Each feature's distribution is plotted in a separate row, with 'Real' news in blue and 'Fake' news in pink.
    """
    # Create a figure with subplots arranged in 3 rows and 1 column
    fig, axes = plt.subplots(
        nrows=len(features), ncols=1, figsize=(15, 6 * len(features))
    )

    for i, feature in enumerate(features):
        # Plot distribution for 'Real' news
        sns.histplot(
            df[df["label"] == 0][feature],
            kde=True,
            color="#66b3ff",
            label="Real",
            ax=axes[i],
        )
        # Plot distribution for 'Fake' news
        sns.histplot(
            df[df["label"] == 1][feature],
            kde=True,
            color="#ff9999",
            label="Fake",
            ax=axes[i],
        )
        # Setting the title for each subplot
        axes[i].set_title(f"{feature} Distribution for {col_name} dataset")
        # Setting the labels for each subplot
        axes[i].set_xlabel(feature)
        axes[i].set_ylabel("Count")
        # Adding legend to each subplot
        axes[i].legend()

    # Adjust layout
    plt.tight_layout()
    plt.show()

In [ ]:
df = pre_process(df)

In [ ]:
df

In [ ]:
df = num_characters(df, "final_news")
df = num_sentences(df, "final_news")
df = num_words(df, "final_news")

In [ ]:
df

In [ ]:
pie_plot(df, "Full Text Article")

In [ ]:
# Plot distribution of the title column
df_title = df

df_title = num_characters(df_title, "title")
df_title = num_sentences(df_title, "title")
df_title = num_words(df_title, "title")

plot_distribution_features(df_title, ["Characters", "Sentences", "Words"], "title")

In [ ]:
# Plot distribution of the text column
df_text = df

df_text = num_characters(df_text, "text")
df_text = num_sentences(df_text, "text")
df_text = num_words(df_text, "text")

plot_distribution_features(df_text, ["Characters", "Sentences", "Words"], "text")

In [ ]:
# Variable Representation for Model
X = df['final_news'].values
y = df['label'].values

# Tokenize the text data
tokenizer = Tokenizer()
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)

# Pad sequences
input_length = max(len(seq) for seq in X)
X = pad_sequences(X, maxlen=input_length)

# Determine vocabulary size
input_dim = len(tokenizer.word_index) + 1

# Define variables
output_dim = 100  # Modify this as per your requirements
params = (input_dim, output_dim, input_length)


[1 points] Choose and explain what metric(s) you will use to evaluate your algorithm’s performance. You should give a detailed argument for why this (these) metric(s) are appropriate on your data. That is, why is the metric appropriate for the task (e.g., in terms of the business case for the task). Please note: rarely is accuracy the best evaluation metric to use. Think deeply about an appropriate measure of performance.


We have decided on the following metrics

Precision: Precision is the ratio of true positive predictions (correctly identifying fake news) to the total number of positive predictions (both true positives and false positives). Precision indicates the algorithm's ability to correctly identify fake news without mistakenly classifying real news as fake. In the context of fake news detection, precision is crucial because incorrectly labeling legitimate news as fake can have serious consequences, including reputational damage to news sources and misinformation propagation.

Recall (Sensitivity): Recall is the ratio of true positive predictions to the total number of actual positive instances (both true positives and false negatives). Recall measures the algorithm's ability to correctly identify all instances of fake news in the dataset. A high recall indicates that the algorithm can effectively detect most of the fake news samples, minimizing the risk of allowing misinformation to spread.

F1 Score: The F1 score is the harmonic mean of precision and recall. It provides a balanced assessment of the algorithm's performance by taking both precision and recall into account. The F1 score is particularly useful when there is an uneven distribution of classes in the dataset, as it considers false positives and false negatives.

Specificity: Specificity is the ratio of true negative predictions (correctly identifying real news) to the total number of actual negative instances (both true negatives and false positives). Specificity measures the algorithm's ability to correctly identify real news without mistakenly classifying it as fake. While precision focuses on the positive class (fake news), specificity provides insights into the algorithm's performance in accurately identifying the negative class (real news).

Area Under the Receiver Operating Characteristic Curve (AUC-ROC): The AUC-ROC is a widely used metric that evaluates the algorithm's overall performance across different threshold settings. It measures the trade-off between the true positive rate (sensitivity) and the false positive rate (1 - specificity) at various classification thresholds. A higher AUC-ROC score indicates better overall performance, considering both true positives and true negatives.

[1 points] Choose the method you will use for dividing your data into training and testing (i.e., are you using Stratified 10-fold cross validation? Shuffle splits? Why?). Explain why your chosen method is appropriate or use more than one method as appropriate. Convince me that your train/test splitting method is a realistic mirroring of how an algorithm would be used in practice. 


We have decided to utilize Stratified 10-fold Cross-Validation:


Handling Class Imbalance: Since your dataset contains an equal distribution of fake news and real news samples, using stratified sampling ensures that each fold of the cross-validation maintains the same proportion of both classes. This is particularly important when dealing with imbalanced datasets, where one class is significantly more prevalent than the other. By maintaining class balance in each fold, the evaluation becomes more representative of the model's performance on unseen data.

Robust Performance Evaluation: Cross-validation provides a more robust estimate of your model's performance by iteratively training and testing the model on different subsets of the data. In Stratified 10-fold Cross-Validation, the dataset is divided into ten equal parts (folds), with each fold serving as the testing set once while the remaining nine folds are used for training. By averaging the performance metrics across all folds, you obtain a more reliable assessment of the model's generalization ability.

Mitigating Overfitting: Cross-validation helps in detecting and mitigating overfitting issues. Overfitting occurs when a model performs well on the training data but fails to generalize to unseen data. With Stratified 10-fold Cross-Validation, the model is exposed to multiple training and testing subsets. If the model consistently performs well across all folds, it indicates that it has learned patterns that are generalizable and not specific to a particular subset.

Optimal Hyperparameter Tuning: Cross-validation is commonly used for hyperparameter tuning. It allows you to evaluate different sets of hyperparameters on the training folds and select the best combination based on the average performance across the testing folds. This helps in finding the optimal hyperparameters that yield the best performance on unseen data.

Realistic Mirroring of Model Usage: Stratified 10-fold Cross-Validation provides a realistic reflection of how the model would be used in practice. It simulates the scenario where the model is trained on a large dataset and tested on unseen data. This approach is especially useful when you don't have a separate validation or holdout dataset for model evaluation.

In [ ]:
kf = KFold(n_splits=2, shuffle=True, random_state=42)

### Modeling

[3 points] Investigate at least two different sequential network architectures (e.g., a CNN and a Transformer). Alternatively, you may also choose a recurrent network and Transformer network. Be sure to use an embedding layer (try to use a pre-trained embedding, if possible). Adjust one hyper-parameter of each network to potentially improve generalization performance (train a total of at least four models). Visualize the performance of training and validation sets versus the training iterations, showing that the models converged.

In [ ]:
# LSTM Model
def lstm_model(input_dim, output_dim, input_length):

    # Define the model architecture
    model = Sequential([
        # Embedding layer
        Embedding(input_dim, output_dim, input_length=input_length),
        
        # LSTM layers
        LSTM(64, return_sequences=True),  # LSTM layer with 64 units and returns sequences
        LSTM(32),  # LSTM layer with 32 units
        
        # Dense output layer
        Dense(1, activation='sigmoid')  # Dense layer with 1 unit and sigmoid activation for binary classification
    ])

    return model

In [ ]:
# GRU Model
def gru_model(input_dim, output_dim, input_length):

    # Define the model architecture
    model = Sequential([
        # Embedding layer
        Embedding(input_dim, output_dim, input_length=input_length),
        
        # GRU layers
        GRU(128, return_sequences=True),  # GRU layer with 128 units and returns sequences
        GRU(64),  # GRU layer with 64 units
        
        # Dense output layer
        Dense(1, activation='sigmoid')  # Dense layer with 1 unit and sigmoid activation for binary classification
    ])

    return model

In [ ]:
def train_and_evaluate_model(model, X_train, y_train, X_test, y_test, epochs=20):
    # Compile the model
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

    # Train the model and store the training history
    history = model.fit(X_train, y_train, epochs=epochs, validation_data=(X_test, y_test), verbose=1)

    # Return the training history and the trained model
    return history, model

In [ ]:
# Plot the training and validation accuracy and loss
def plot_history(history, title):
    plt.figure(figsize=(12, 4))
    plt.suptitle(title)

    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Training Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend()

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Training Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()

    plt.show()


In [44]:
# Train and evaluate all model variations

epochs = 10

model_variations = [
    (lstm_model, params),
    (gru_model, params),
]
histories = []
models = []

for train_index, test_index in kf.split(X, y):
    print("Fold Number:", len(histories) + 1)
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]

    sub_histories = []
    sub_models = []
    for model_func, variation in model_variations:
        model = model_func(*variation)
        history, model = train_and_evaluate_model(model, X_train=X_train, y_train=y_train, X_test=X_test, y_test=y_test, epochs=epochs)
        sub_histories.append(history)
        sub_models.append(model)

    histories.append(sub_histories)
    models.append(sub_models)

# Visualize the performance of all model variations
for idx, (model_func, variation) in enumerate(model_variations):
    title = f"{model_func.__name__} Variation {variation}"
    for i, history in enumerate(histories):
        plot_history(history[i][idx], title=f"{title} - Fold {i+1}")

KeyboardInterrupt: 

[1 points] Using the best parameters and architecture from the Transformer in the previous step, add a second Multi-headed self attention layer to your network. That is, the input to the second attention layer should be the output sequence of the first attention layer.  Visualize the performance of training and validation sets versus the training iterations, showing that the model converged.. 


In [ ]:
# Code

[2 points] Use the method of train/test splitting and evaluation criteria that you argued for at the beginning of the lab. Visualize the results of all the models you trained.  Use proper statistical comparison techniques to determine which method(s) is (are) superior.  


In [ ]:
# Code

### Exceptional Work

Use the pre-trained ConceptNet Numberbatch embedding and compare to pre-trained GloVe. Which method is better for your specific application? 


In [ ]:
# Code for pre-trained ConceptNet Numberbatch embedding and pre-trained GloVe